In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
# do pip install py-openaq in the pandas environment
import openaq
from pprint import pprint


In [7]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [8]:
engine = create_engine("sqlite:///output_data/airquality.sqlite")

In [9]:
# api = openaq.OpenAQ()
# res = api.measurements(city='St. Louis', parameter='pm25', limit=10000, df=True)
# res = res.query("value >= 0.0")
# # g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)
# # g.map(plt.hist, "value")
# # print(res)
# res.dtypes
# stl_df = pd.DataFrame(res)
# stl_df

api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='pm25', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_pm25 = pd.DataFrame(res)
us_df_pm25.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-15 19:00:00,Freemansburg,pm25,14.0,b'\xc2\xb5g/m\xc2\xb3',US,Allentown-Bethlehem-Easton,2019-11-16 00:00:00+00:00,40.6281,-75.3411
2019-11-15 19:00:00,Tioga County,pm25,14.8,b'\xc2\xb5g/m\xc2\xb3',US,TIOGA,2019-11-16 00:00:00+00:00,41.6447,-76.9392
2019-11-15 19:00:00,Charleroi,pm25,17.1,b'\xc2\xb5g/m\xc2\xb3',US,Pittsburgh,2019-11-16 00:00:00+00:00,40.1469,-79.9033
2019-11-15 19:00:00,Florence,pm25,21.6,b'\xc2\xb5g/m\xc2\xb3',US,Pittsburgh,2019-11-16 00:00:00+00:00,40.4447,-80.4200
2019-11-15 19:00:00,Houston,pm25,18.1,b'\xc2\xb5g/m\xc2\xb3',US,WASHINGTON,2019-11-16 00:00:00+00:00,40.2690,-80.2440


In [10]:
pm25_groupby = us_df_pm25.groupby('location')
pm25_first = pm25_groupby.first()

pm25mean = us_df_pm25.groupby('location').mean()
pm25mean = pm25mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

pm25 = pd.merge(pm25_first, pm25mean, on = 'location')
pm25 = pm25.drop(['value','lat','lng'], axis=1)

pm25.to_sql(name='pm25', con=engine)

pm25.head()

,parameter,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude,avgvalue
location,,,,,,,,
2912 Coffey,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Omaha-Council Bluffs,2019-11-15 23:00:00+00:00,41.133317,-95.956078,10.785714
ADEQ-DSR,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Little Rock-North Little Rock,2019-11-15 21:00:00+00:00,34.681100,-92.328300,7.385714
AIR LAB,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Knoxville,2019-11-15 23:00:00+00:00,35.980300,-83.930600,8.607692
ALLEN PARK,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Detroit-Warren-Livonia,2019-11-15 23:00:00+00:00,42.228300,-83.209200,9.014286
APOLLO BEACH,pm25,b'\xc2\xb5g/m\xc2\xb3',US,HILLSBOROUGH,2019-11-15 23:00:00+00:00,27.779712,-82.419835,3.778571
APPLETON AAL,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Appleton,2019-11-15 23:00:00+00:00,44.306900,-88.395600,14.083333
ARNOLD,pm25,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-15 13:00:00+00:00,38.448581,-90.398436,12.625000
ARNOLD WEST,pm25,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-16 00:00:00+00:00,38.448581,-90.398436,11.716667
ATHENS,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Athens,2019-11-15 23:00:00+00:00,35.451100,-84.599200,6.000000


In [11]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='bc', limit=9999, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_bc = pd.DataFrame(res)
us_df_bc.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-14 17:00:00,Berkeley Aquatic Par,bc,0.83,b'\xc2\xb5g/m\xc2\xb3',US,ALAMEDA,2019-11-15 01:00:00+00:00,37.864767,-122.302741
2019-11-14 17:00:00,Laney College,bc,1.04,b'\xc2\xb5g/m\xc2\xb3',US,San Francisco-Oakland-Fremont,2019-11-15 01:00:00+00:00,37.793624,-122.263376
2019-11-14 19:00:00,Evansville-Buena Vis,bc,0.91,b'\xc2\xb5g/m\xc2\xb3',US,Evansville,2019-11-15 01:00:00+00:00,38.013300,-87.577200
2019-11-14 20:00:00,Howard County Near R,bc,0.40,b'\xc2\xb5g/m\xc2\xb3',US,HOWARD,2019-11-15 01:00:00+00:00,39.143197,-76.846192
2019-11-14 19:00:00,Gary-IITRI,bc,0.56,b'\xc2\xb5g/m\xc2\xb3',US,Chicago-Naperville-Joliet,2019-11-15 01:00:00+00:00,41.606700,-87.304700
2019-11-14 20:00:00,Elkhart-Prairie St.,bc,7.13,b'\xc2\xb5g/m\xc2\xb3',US,Elkhart-Goshen,2019-11-15 01:00:00+00:00,41.656944,-85.968330
2019-11-14 20:00:00,E Providence,bc,1.84,b'\xc2\xb5g/m\xc2\xb3',US,Providence-New Bedford-Fall River,2019-11-15 01:00:00+00:00,41.840302,-71.361702
2019-11-14 20:00:00,Near Road,bc,3.05,b'\xc2\xb5g/m\xc2\xb3',US,Providence-New Bedford-Fall River,2019-11-15 01:00:00+00:00,41.829495,-71.417457
2019-11-14 17:00:00,San Jose - Knox Ave,bc,0.69,b'\xc2\xb5g/m\xc2\xb3',US,San Jose-Sunnyvale-Santa Clara,2019-11-15 01:00:00+00:00,37.338202,-121.849892


In [12]:
bc_groupby = us_df_bc.groupby('location')
bc_first = bc_groupby.first()

bcmean = us_df_bc.groupby('location').mean()
bcmean = bcmean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

bc = pd.merge(bc_first, bcmean, on = 'location')
bc = bc.drop(['value','lat','lng'], axis=1)

bc.to_sql(name='bc', con=engine)

bc.head()

,parameter,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude,avgvalue
location,,,,,,,,
Arden Arcade - Del P,bc,b'\xc2\xb5g/m\xc2\xb3',US,Sacramento--Arden-Arcade--Roseville,2019-11-15 01:00:00+00:00,38.613804,-121.368007,0.823432
Berkeley Aquatic Par,bc,b'\xc2\xb5g/m\xc2\xb3',US,ALAMEDA,2019-11-15 01:00:00+00:00,37.864767,-122.302741,1.289099
E Providence,bc,b'\xc2\xb5g/m\xc2\xb3',US,Providence-New Bedford-Fall River,2019-11-15 01:00:00+00:00,41.840302,-71.361702,0.400203
Elkhart-Prairie St.,bc,b'\xc2\xb5g/m\xc2\xb3',US,Elkhart-Goshen,2019-11-15 01:00:00+00:00,41.656944,-85.968330,4.071962
Evansville-Buena Vis,bc,b'\xc2\xb5g/m\xc2\xb3',US,Evansville,2019-11-15 01:00:00+00:00,38.013300,-87.577200,0.596353
Gary-IITRI,bc,b'\xc2\xb5g/m\xc2\xb3',US,Chicago-Naperville-Joliet,2019-11-15 01:00:00+00:00,41.606700,-87.304700,0.511559
HU-Beltsville,bc,b'\xc2\xb5g/m\xc2\xb3',US,Washington-Arlington-Alexandria,2019-11-15 01:00:00+00:00,39.055302,-76.878304,0.450820
Howard County Near R,bc,b'\xc2\xb5g/m\xc2\xb3',US,HOWARD,2019-11-15 01:00:00+00:00,39.143197,-76.846192,1.143678
Indpls - I-70 E,bc,b'\xc2\xb5g/m\xc2\xb3',US,Indianapolis,2019-11-15 01:00:00+00:00,39.787933,-86.130880,0.758121


In [13]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='pm10', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_pm10 = pd.DataFrame(res)
us_df_pm10.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-15 16:00:00,Mesa 2 /Nipomo,pm10,97,b'\xc2\xb5g/m\xc2\xb3',US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.020790,-120.563890
2019-11-15 16:00:00,Nipomo Mesa CDF,pm10,75,b'\xc2\xb5g/m\xc2\xb3',US,SAN LUIS OBISPO,2019-11-16 00:00:00+00:00,35.046760,-120.587770
2019-11-15 16:00:00,Nipomo RP,pm10,44,b'\xc2\xb5g/m\xc2\xb3',US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.031500,-120.501010
2019-11-15 16:00:00,Atascadero2,pm10,37,b'\xc2\xb5g/m\xc2\xb3',US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.494556,-120.666203
2019-11-15 17:00:00,Ute 3,pm10,41,b'\xc2\xb5g/m\xc2\xb3',US,Durango,2019-11-16 00:00:00+00:00,37.102580,-107.870219
2019-11-15 18:00:00,St. Joseph,pm10,38,b'\xc2\xb5g/m\xc2\xb3',US,St. Joseph,2019-11-16 00:00:00+00:00,39.741667,-94.858333
2019-11-15 18:00:00,Hillcrest HS,pm10,29,b'\xc2\xb5g/m\xc2\xb3',US,Springfield,2019-11-16 00:00:00+00:00,37.256069,-93.299692
2019-11-15 18:00:00,Front Street,pm10,145,b'\xc2\xb5g/m\xc2\xb3',US,Kansas City,2019-11-16 00:00:00+00:00,39.131980,-94.531280
2019-11-15 18:00:00,CARTHAGE,pm10,19,b'\xc2\xb5g/m\xc2\xb3',US,Joplin,2019-11-16 00:00:00+00:00,37.198220,-94.317020


In [14]:
pm10_groupby = us_df_pm10.groupby('location')
pm10_first = pm10_groupby.first()

pm10mean = us_df_pm10.groupby('location').mean()
pm10mean = pm10mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

pm10 = pd.merge(pm10_first, pm10mean, on = 'location')
pm10 = pm10.drop(['value','lat','lng'], axis=1)

pm10.to_sql(name='pm10', con=engine)

pm10.head()

,parameter,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude,avgvalue
location,,,,,,,,
7E Deming Airport,pm10,b'\xc2\xb5g/m\xc2\xb3',US,Deming,2019-11-16 00:00:00+00:00,32.255800,-107.722800,17.243243
ARNOLD,pm10,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-15 13:00:00+00:00,38.448581,-90.398436,14.920000
ARNOLD WEST,pm10,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-16 00:00:00+00:00,38.448581,-90.398436,18.000000
Ajo,pm10,b'\xc2\xb5g/m\xc2\xb3',US,PIMA,2019-11-16 00:00:00+00:00,32.382034,-112.857452,37.171429
Alamo Lake,pm10,b'\xc2\xb5g/m\xc2\xb3',US,LA PAZ,2019-11-16 00:00:00+00:00,34.243889,-113.558611,14.486486
Anaheim,pm10,b'\xc2\xb5g/m\xc2\xb3',US,Los Angeles-Long Beach-Santa Ana,2019-11-16 00:00:00+00:00,33.830586,-117.938509,31.540541
Anthony,pm10,b'\xc2\xb5g/m\xc2\xb3',US,Las Cruces,2019-11-16 00:00:00+00:00,32.003600,-106.599200,47.516129
Apache Junction,pm10,b'\xc2\xb5g/m\xc2\xb3',US,Phoenix-Mesa-Scottsdale,2019-11-16 00:00:00+00:00,33.420000,-111.504200,24.868421
Atascadero2,pm10,b'\xc2\xb5g/m\xc2\xb3',US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.494556,-120.666203,18.578947


In [15]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='so2', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_so2 = pd.DataFrame(res)
us_df_so2.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-15 16:00:00,Mesa 2 /Nipomo,so2,0.000,ppm,US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.020790,-120.563890
2019-11-15 19:00:00,NCORE,so2,0.001,ppm,US,BROWARD,2019-11-16 00:00:00+00:00,26.054047,-80.257608
2019-11-15 18:00:00,Jackson NCORE,so2,0.001,ppm,US,Jackson,2019-11-16 00:00:00+00:00,32.329111,-90.182722
2019-11-15 18:00:00,Buick NE,so2,0.006,ppm,US,IRON,2019-11-16 00:00:00+00:00,37.652142,-91.116530
2019-11-15 18:00:00,Troost,so2,0.002,ppm,US,Kansas City,2019-11-16 00:00:00+00:00,39.104650,-94.570550
2019-11-15 18:00:00,"Herculaneum, Mott St",so2,0.001,ppm,US,St. Louis,2019-11-16 00:00:00+00:00,38.263394,-90.379667
2019-11-15 18:00:00,Rider Trail I-70,so2,0.002,ppm,US,ST. LOUIS,2019-11-16 00:00:00+00:00,38.752640,-90.448840
2019-11-15 18:00:00,BLAIR STREET,so2,0.001,ppm,US,St. Louis,2019-11-16 00:00:00+00:00,38.656449,-90.198548
2019-11-15 18:00:00,Sheldon Station,so2,0.000,ppm,US,LANCASTER,2019-11-16 00:00:00+00:00,40.554760,-96.780350


In [16]:
so2_groupby = us_df_so2.groupby('location')
so2_first = so2_groupby.first()

so2mean = us_df_so2.groupby('location').mean()
so2mean = so2mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

so2 = pd.merge(so2_first, so2mean, on = 'location')
so2 = so2.drop(['value','lat','lng'], axis=1)

so2.to_sql(name='so2', con=engine)

so2.head()

,parameter,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude,avgvalue
location,,,,,,,,
16th and Whitmore,so2,ppm,US,Omaha-Council Bluffs,2019-11-16 00:00:00+00:00,41.322472,-95.937992,0.000316
ALLEN PARK,so2,ppm,US,Detroit-Warren-Livonia,2019-11-16 00:00:00+00:00,42.228300,-83.209200,0.001022
APOLLO BEACH,so2,ppm,US,HILLSBOROUGH,2019-11-16 00:00:00+00:00,27.779712,-82.419835,0.000761
Acadia NP - McFarlan,so2,ppm,US,HANCOCK,2019-11-15 23:00:00+00:00,44.377048,-68.260902,0.000000
Akron NIFH/STEM,so2,ppm,US,Akron,2019-11-16 00:00:00+00:00,41.079140,-81.516270,0.000000
Amanda Elementary,so2,ppm,US,BUTLER,2019-11-16 00:00:00+00:00,39.478849,-84.407675,0.000500
Anacortes-202 Ave (S,so2,ppm,US,SKAGIT,2019-11-15 23:00:00+00:00,48.520590,-122.614280,0.000956
Andrew Johnson Elem.,so2,ppm,US,Kingsport-Bristol-Bristol,2019-11-15 23:00:00+00:00,36.526416,-82.528679,0.002000
Arden Arcade - Del P,so2,ppm,US,Sacramento--Arden-Arcade--Roseville,2019-11-16 00:00:00+00:00,38.613804,-121.368007,0.000047


In [17]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='no2', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_no2 = pd.DataFrame(res)
us_df_no2.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-15 16:00:00,Nipomo RP,no2,0.002,ppm,US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.031500,-120.501010
2019-11-15 16:00:00,Atascadero2,no2,0.003,ppm,US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.494556,-120.666203
2019-11-15 17:00:00,Ute 1,no2,0.008,ppm,US,Durango,2019-11-16 00:00:00+00:00,37.136780,-107.628630
2019-11-15 17:00:00,Ute 3,no2,0.008,ppm,US,Durango,2019-11-16 00:00:00+00:00,37.102580,-107.870219
2019-11-15 19:00:00,Near Road,no2,0.017,ppm,US,BROWARD,2019-11-16 00:00:00+00:00,26.131256,-80.167847
2019-11-15 19:00:00,DANIA_BEACH,no2,0.026,ppm,US,Miami-Fort Lauderdale-Miami Beach,2019-11-16 00:00:00+00:00,26.086400,-80.111400
2019-11-15 18:00:00,Troost,no2,0.032,ppm,US,Kansas City,2019-11-16 00:00:00+00:00,39.104650,-94.570550
2019-11-15 18:00:00,Blue Ridge I-70,no2,0.033,ppm,US,Kansas City,2019-11-16 00:00:00+00:00,39.047911,-94.450513
2019-11-15 18:00:00,MTSP,no2,0.003,ppm,US,MONROE,2019-11-16 00:00:00+00:00,39.475100,-91.788990


In [18]:
no2_groupby = us_df_no2.groupby('location')
no2_first = no2_groupby.first()

no2mean = us_df_no2.groupby('location').mean()
no2mean = no2mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

no2 = pd.merge(no2_first, no2mean, on = 'location')
no2 = no2.drop(['value','lat','lng'], axis=1)

no2.to_sql(name='no2', con=engine)

no2.head()

,parameter,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude,avgvalue
location,,,,,,,,
60 Near Road,no2,ppm,US,SAN BERNARDINO,2019-11-16 00:00:00+00:00,34.030900,-117.617403,0.037674
710 Near Road,no2,ppm,US,LOS ANGELES,2019-11-16 00:00:00+00:00,33.859662,-118.200707,0.035140
AURORA HILLS,no2,ppm,US,Washington-Arlington-Alexandria,2019-11-15 23:00:00+00:00,38.859402,-77.058899,0.023805
Anaheim,no2,ppm,US,Los Angeles-Long Beach-Santa Ana,2019-11-16 00:00:00+00:00,33.830586,-117.938509,0.018707
Anaheim Near Road,no2,ppm,US,ORANGE,2019-11-16 00:00:00+00:00,33.819305,-117.918759,0.023500
Arden Arcade - Del P,no2,ppm,US,Sacramento--Arden-Arcade--Roseville,2019-11-16 00:00:00+00:00,38.613804,-121.368007,0.007600
Atascadero2,no2,ppm,US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.494556,-120.666203,0.005415
Azalea,no2,ppm,US,Tampa-St. Petersburg-Clearwater,2019-11-15 23:00:00+00:00,27.785800,-82.740000,0.005182
Azusa,no2,ppm,US,Los Angeles-Long Beach-Santa Ana,2019-11-16 00:00:00+00:00,34.136474,-117.923963,0.025140


In [19]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='o3', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_o3 = pd.DataFrame(res)
us_df_o3.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-15 19:00:00,Freemansburg,o3,0.002,ppm,US,Allentown-Bethlehem-Easton,2019-11-16 00:00:00+00:00,40.628100,-75.341100
2019-11-15 19:00:00,Tioga County,o3,0.027,ppm,US,TIOGA,2019-11-16 00:00:00+00:00,41.644700,-76.939200
2019-11-15 19:00:00,Charleroi,o3,0.011,ppm,US,Pittsburgh,2019-11-16 00:00:00+00:00,40.146900,-79.903300
2019-11-15 19:00:00,Florence,o3,0.016,ppm,US,Pittsburgh,2019-11-16 00:00:00+00:00,40.444700,-80.420000
2019-11-15 19:00:00,Houston,o3,0.016,ppm,US,WASHINGTON,2019-11-16 00:00:00+00:00,40.269000,-80.244000
2019-11-15 19:00:00,York,o3,0.001,ppm,US,York-Hanover,2019-11-16 00:00:00+00:00,39.965300,-76.699400
2019-11-15 16:00:00,Morro Bay,o3,0.033,ppm,US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.366390,-120.842600
2019-11-15 16:00:00,Nipomo RP,o3,0.036,ppm,US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.031500,-120.501010
2019-11-15 16:00:00,Atascadero2,o3,0.036,ppm,US,San Luis Obispo-Paso Robles,2019-11-16 00:00:00+00:00,35.494556,-120.666203


In [20]:
o3_groupby = us_df_o3.groupby('location')
o3_first = o3_groupby.first()

o3mean = us_df_o3.groupby('location').mean()
o3mean = o3mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

o3 = pd.merge(o3_first, o3mean, on = 'location')
o3 = o3.drop(['value','lat','lng'], axis=1)

o3.to_sql(name='o3', con=engine)

o3.head()

,parameter,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude,avgvalue
location,,,,,,,,
22nd Street & Craycr,o3,ppm,US,Tucson,2019-11-16 00:00:00+00:00,32.205000,-110.877200,0.034214
2LL Los Lunas,o3,ppm,US,Albuquerque,2019-11-16 00:00:00+00:00,34.814700,-106.739700,0.028286
AJ Maintenance Yard,o3,ppm,US,Phoenix-Mesa-Scottsdale,2019-11-16 00:00:00+00:00,33.420601,-111.503304,0.042214
ALLEN PARK,o3,ppm,US,Detroit-Warren-Livonia,2019-11-16 00:00:00+00:00,42.228300,-83.209200,0.012929
ALSIP,o3,ppm,US,Chicago-Naperville-Joliet,2019-11-15 22:00:00+00:00,41.670800,-87.732500,0.005833
Abington,o3,ppm,US,Willimantic,2019-11-15 23:00:00+00:00,41.840200,-72.010000,0.031308
Acadia NP - McFarlan,o3,ppm,US,HANCOCK,2019-11-15 23:00:00+00:00,44.377048,-68.260902,0.029600
Alabama-Coushatta,o3,ppm,US,POLK,2019-11-15 19:00:00+00:00,30.668100,-94.673900,0.040000
Alamo Lake,o3,ppm,US,LA PAZ,2019-11-16 00:00:00+00:00,34.243889,-113.558611,0.037733


In [21]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='co', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_co = pd.DataFrame(res)
us_df_co.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-15 17:00:00,Ute 1,co,0.14,ppm,US,Durango,2019-11-16 00:00:00+00:00,37.136780,-107.628630
2019-11-15 19:00:00,NCORE,co,0.23,ppm,US,BROWARD,2019-11-16 00:00:00+00:00,26.054047,-80.257608
2019-11-15 19:00:00,Near Road,co,0.38,ppm,US,BROWARD,2019-11-16 00:00:00+00:00,26.131256,-80.167847
2019-11-15 18:00:00,Jackson NCORE,co,0.12,ppm,US,Jackson,2019-11-16 00:00:00+00:00,32.329111,-90.182722
2019-11-15 18:00:00,Blue Ridge I-70,co,0.62,ppm,US,Kansas City,2019-11-16 00:00:00+00:00,39.047911,-94.450513
2019-11-15 18:00:00,BLAIR STREET,co,0.83,ppm,US,St. Louis,2019-11-16 00:00:00+00:00,38.656449,-90.198548
2019-11-15 18:00:00,Forest Park,co,0.41,ppm,US,St. Louis,2019-11-16 00:00:00+00:00,38.631057,-90.281144
2019-11-15 17:00:00,Converse County Long,co,0.07,ppm,US,CONVERSE,2019-11-16 00:00:00+00:00,43.100000,-105.500000
2019-11-15 17:00:00,Phoenix JLG Supersit,co,0.49,ppm,US,Phoenix-Mesa-Scottsdale,2019-11-16 00:00:00+00:00,33.503601,-112.095001


In [22]:
co_groupby = us_df_co.groupby('location')
co_first = co_groupby.first()

comean = us_df_co.groupby('location').mean()
comean = comean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

co = pd.merge(co_first, comean, on = 'location')
co = co.drop(['value','lat','lng'], axis=1)

co.to_sql(name='co', con=engine)

co.head()

,parameter,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude,avgvalue
location,,,,,,,,
77th and Dodge,co,ppm,US,Omaha-Council Bluffs,2019-11-16 00:00:00+00:00,41.259478,-96.034714,0.387759
ALLEN PARK,co,ppm,US,Detroit-Warren-Livonia,2019-11-16 00:00:00+00:00,42.228300,-83.209200,0.000000
AURORA HILLS,co,ppm,US,Washington-Arlington-Alexandria,2019-11-15 23:00:00+00:00,38.859402,-77.058899,0.465517
Anaheim,co,ppm,US,Los Angeles-Long Beach-Santa Ana,2019-11-16 00:00:00+00:00,33.830586,-117.938509,0.395167
Anaheim Near Road,co,ppm,US,ORANGE,2019-11-16 00:00:00+00:00,33.819305,-117.918759,0.684262
Arden Arcade - Del P,co,ppm,US,Sacramento--Arden-Arcade--Roseville,2019-11-16 00:00:00+00:00,38.613804,-121.368007,0.362632
Azusa,co,ppm,US,Los Angeles-Long Beach-Santa Ana,2019-11-16 00:00:00+00:00,34.136474,-117.923963,0.536129
BLAIR STREET,co,ppm,US,St. Louis,2019-11-16 00:00:00+00:00,38.656449,-90.198548,0.354828
Bakersfield-Muni,co,ppm,US,Bakersfield,2019-11-15 23:00:00+00:00,35.331300,-119.000000,0.387966


In [53]:
# api = openaq.OpenAQ()
# res2 = api.locations(coordinates='38.63, -90.20', radius='50000', limit=10000, df=True)

# # res2 = res2.query("value >= 0.0")
# # g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# # g.map(plt.hist, "value")
# # print(res)
# res.dtypes

# stl_df2 = pd.DataFrame(res2)
# stl_df2